In [4]:
import os
import json
import io
from tqdm import tqdm

In [75]:
def create_dictionary(lyrics):
    save_lyrics = False
    last_structure_name = None
    dictionary = {}
    reading_title = False
    reading_block = False
    block = ""
    title = ""
    titles_list = []
    block_list = []
    for char in tqdm(lyrics):
        if char == "[":
            if reading_block:
                titles_list.append(title)
                block_list.append(block)
            reading_block = False
            reading_title = True
            title = ""
            block = ""
            continue
        if char == "]":
            reading_title = False
            reading_block, title = is_important(title)
            continue
        if reading_title:
            title += char
        if reading_block:
            block += char
    if reading_block:
        titles_list.append(title)
        block_list.append(block)
    return titles_list, block_list


def is_important(title):
    if title[:5].lower() in SONG_STRUCTURE:
        title = title[:5]
        return True, title
    elif title[:6].lower() in SONG_STRUCTURE:
        title = title[:6]
        return True, title
    return False, title


def cleaning(titles, blocks):
    cleaned_titles, cleaned_blocks = [], []
    for title, block in zip(titles, blocks):
        cleaned_titles.append(title.strip())
        cleaned_blocks.append(block.strip())
    return cleaned_titles, cleaned_blocks

def save_data(save_path, data):
    with open(save_path, "w", encoding="utf-8") as write_file:
        write_file.write(data)
        
def clean_underscore(sentence):
    """
    Removes underscores from a sentence
    """
    sentence = sentence.split()
    new_sentence = []
    for word in sentence:
        if len(word) >= 2:
            if word[0] == "_" and word[-1] == "_":
                word = word[1:]
                word = word[:-1]
        new_sentence.append(word)
    new_sentence.append("")
        
    return " ".join(new_sentence)

In [18]:
SONG_STRUCTURE = ["intro", "chorus", "verse", "bridge", "outro"]  # structures we are looking for
songs_file_path = "raw_songs_nas/"
books_file_path = "books/"
save_path = "data.txt"
NB_SONGS = 200
NB_BOOKS = 2
data = ""

song_files_list = os.listdir("./raw_songs_nas")
book_files_list = os.listdir("./books")
for idx, song in enumerate(song_files_list):
    # if we don't want to take all the available songs
    if idx == NB_SONGS:  
        break
        
    with open(songs_file_path + song, "r") as json_file:
        raw_data = json.load(json_file)
        for item in raw_data["songs"]:  # we get the song
            lyric = item["lyrics"]  # we are only interested in the lyrics
            titles, blocks = create_dictionary(lyric)
            assert(len(titles) == len(blocks))
            titles, blocks = cleaning(titles, blocks)
            cleaned_blocks = []
            for block in blocks:
                temp = block.replace("\n\n", " END_TOKEN ")
                temp = temp.replace("\n", " END_TOKEN ")
                temp = temp.replace("\"", "")  # we get rid of " "
                #temp = temp.replace("\'", "")
                temp += (" END_TOKEN ")  # end of song structure
                cleaned_blocks.append(temp)
    for lyric in cleaned_blocks:
        data += lyric

data = data.lower()  # we get the text in lowercase
print(data)
#save_data(save_path, data)  # we save the data.

100%|█████████████████████████████████████████████████████████████████████████| 2360/2360 [00:00<00:00, 1182058.45it/s]


100%|█████████████████████████████████████████████████████████████████████████| 4876/4876 [00:00<00:00, 1634544.94it/s]


100%|█████████████████████████████████████████████████████████████████████████| 3468/3468 [00:00<00:00, 1738893.76it/s]


what up? end_token escobar season begins end_token repent your sins end_token nine-six end_token trackmasters end_token check it end_token spark the lye with the eight-forty i fly end_token pretty thug show no love its like when doves cry end_token maybe i'm just too demanding, parked the land end_token mark buchanan, fuck a job i'mma die scramblin end_token digger gold, dig a bigger hole in ya soul end_token que pasa, pablo's throne cabron end_token at the airport the mobb picked me up in a truck end_token jewelry chunky like fuck never scared to get stuck end_token so whats the deal papi? end_token heard the feds could of knocked me end_token had the cuban posse all up in my room and lobby end_token negotiating, my name holds weight, we motivatin' end_token hustling, as if god was dancing with satan end_token the firm waitin', to get ajourned cause we facing end_token double life, thuggin for life, you taste it end_token the white numbin ya toungue end_token i bought it from dominica

In [77]:
book_files_list = os.listdir("./books")
data_book = ""
for idx, book in enumerate(book_files_list):
    cleaned_book = []
    
    # if we don't want to take all the available books
    if idx == NB_BOOKS:  
        break
        
    with open(books_file_path + book, "r") as txt_file:
        lines = txt_file.readlines()
        for line in lines:
            line = clean_underscore(line)
            temp = line.replace(".", " END_TOKEN")
            temp = temp.replace("\n\n", " ")
            temp = temp.replace("--", " ")
            cleaned_book.append(temp)
    for lyric in cleaned_book:
        data_book += lyric

data_book = data_book.lower()  # we get the text in lowercase
print(data_book)

ï»¿the subject of this essay is not the so-called liberty of the will, so unfortunately opposed to the misnamed doctrine of philosophical necessity; but civil, or social liberty: the nature and limits of the power which can be legitimately exercised by society over the individual end_token a question seldom stated, and hardly ever discussed, in general terms, but which profoundly influences the practical controversies of the age by its latent presence, and is likely soon to make itself recognised as the vital question of the future end_token it is so far from being new, that in a certain sense, it has divided mankind, almost from the remotest ages; but in the stage of progress into which the more civilised portions of the species have now entered, it presents itself under new conditions, and requires a different and more fundamental treatment end_token the struggle between liberty and authority is the most conspicuous feature in the portions of history with which we are earliest famili

In [72]:
print(data+data_book)

what up? end_token escobar season begins end_token repent your sins end_token nine-six end_token trackmasters end_token check it end_token spark the lye with the eight-forty i fly end_token pretty thug show no love its like when doves cry end_token maybe i'm just too demanding, parked the land end_token mark buchanan, fuck a job i'mma die scramblin end_token digger gold, dig a bigger hole in ya soul end_token que pasa, pablo's throne cabron end_token at the airport the mobb picked me up in a truck end_token jewelry chunky like fuck never scared to get stuck end_token so whats the deal papi? end_token heard the feds could of knocked me end_token had the cuban posse all up in my room and lobby end_token negotiating, my name holds weight, we motivatin' end_token hustling, as if god was dancing with satan end_token the firm waitin', to get ajourned cause we facing end_token double life, thuggin for life, you taste it end_token the white numbin ya toungue end_token i bought it from dominica

 end_token ï»¿the subject of this essay is not the so-called liberty of the will, sounfortunately opposed to the misnamed doctrine of philosophicalnecessity; but civil, or social liberty: the nature and limits of thepower which can be legitimately exercised by society over theindividual end_token a question seldom stated, and hardly ever discussed, ingeneral terms, but which profoundly influences the practicalcontroversies of the age by its latent presence, and is likely soon tomake itself recognised as the vital question of the future end_token it is so farfrom being new, that in a certain sense, it has divided mankind, almostfrom the remotest ages; but in the stage of progress into which the morecivilised portions of the species have now entered, it presents itselfunder new conditions, and requires a different and more fundamentaltreatment end_tokenthe struggle between liberty and authority is the most conspicuousfeature in the portions of history with which we are earliest familiar,